# Assignment 3 - Association Rule Mining

## Instructions

For the Third Assignment, we are going to be performing association rule mining with Apriori and FP-Growth algorithm.

1. There are set of questions associated with each task for apriori and fp-growth respectively.
2. The dataset used for whole assignment is same.
3. Each Task is presenting a challenge.

### GROUP 5

#### First Student Name and ID: Kavish Khatri(1002069464)
#### Second Student Name and ID: Mounika Kanneti(1002032500)
#### Third Student Name and ID: Vijitha Kotapati(1001860730)


Simply run the blow cell to load the dataset.

The dataset has 38765 rows of the purchase orders of people from the grocery stores.

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori,fpgrowth, association_rules

# Load the dataset
data = pd.read_csv("Data_batch1.csv")
data

Member_number        Date        itemDescription
0               1808  21-07-2015         tropical fruit
1               2552  05-01-2015             whole milk
2               2300  19-09-2015              pip fruit
3               1187  12-12-2015       other vegetables
4               3037  01-02-2015             whole milk
...              ...         ...                    ...
38760           4471  08-10-2014          sliced cheese
38761           2022  23-02-2014                  candy
38762           1097  16-04-2014               cake bar
38763           1510  03-12-2014  fruit/vegetable juice
38764           1521  26-12-2014               cat food

[38765 rows x 3 columns]

### Task A: 5 points

1. Print details on data.
2. Check unique items and modify items if found any misfit names.

In [2]:
print('1. Shape of Dataset:', data.shape)
print('\n2. Columns:', data.columns)
data.info()
print('\n3. First few rows:')
data_head = data.head()
data_head

1. Shape of Dataset: (38765, 3)

2. Columns: Index(['Member_number', 'Date', 'itemDescription'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB

3. First few rows:


Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-12-2015  other vegetables
4           3037  01-02-2015        whole milk

We need to inspect the data and identify any inconsistencies in the item names to check for unique items and modify them if any misfit names are found. After identifying the misfit names, we can appropriately modify the item names to fix them.

In this code below we modified the item names to lowercase using the str.lower() method. This ensures that all product names are represented consistently in lowercase.

In [3]:
# Check for unique items 
unique_items_original_data = data['itemDescription'].unique()
print("\n1. Unique Items in Original Dataset:")
print(unique_items_original_data)

# Identify misfit names if there are any
misfit_items_data = set()
for item_lst in unique_items_original_data:
    for item in item_lst.split(', '):
           if item.lower() not in unique_items_original_data:
                   misfit_items_data.add(item)
            
if misfit_items_data:
    print("\n2. Misfit Names Found:", misfit_items_data)
    
# Modify misfit names
# For example, if "soda" and "Soda" are considered different, we can change all the item names to lowercase.
data['itemDescription'] = data['itemDescription'].str.lower()

# After modification, check unique items again
unique_items_modified = data['itemDescription'].unique()
print("\n3. Unique Items (Modified):")
print(unique_items_modified)


1. Unique Items in Original Dataset:
['tropical fruit' 'whole milk' 'pip fruit' 'other vegetables' 'rolls/buns'
 'pot plants' 'citrus fruit' 'beef' 'frankfurter' 'chicken' 'butter'
 'fruit/vegetable juice' 'packaged fruit/vegetables' 'chocolate'
 'specialty bar' 'butter milk' 'bottled water' 'yogurt' 'sausage'
 'brown bread' 'hamburger meat' 'root vegetables' 'pork' 'pastry'
 'canned beer' 'berries' 'coffee' 'misc. beverages' 'ham' 'turkey'
 'curd cheese' 'red/blush wine' 'frozen potato products' 'flour' 'sugar'
 'frozen meals' 'herbs' 'soda' 'detergent' 'grapes' 'processed cheese'
 'fish' 'sparkling wine' 'newspapers' 'curd' 'pasta' 'popcorn'
 'finished products' 'beverages' 'bottled beer' 'dessert' 'dog food'
 'specialty chocolate' 'condensed milk' 'cleaner' 'white wine' 'meat'
 'ice cream' 'hard cheese' 'cream cheese ' 'liquor' 'pickled vegetables'
 'liquor (appetizer)' 'UHT-milk' 'candy' 'onions' 'hair spray'
 'photo/film' 'domestic eggs' 'margarine' 'shopping bags' 'salt' 'oil'
 

### Task B: 7 points

Create a list of lists with grouping on member number and the transaction date and store it in a dataframe

This code snippet groups transaction data by 'Member_number' and 'Date', aggregates the items purchased in each transaction into lists, creates a new DataFrame to store the grouped data, displays the shape of the DataFrame, and provides a preview of the first few rows of the grouped data.

In [4]:
# Create a list of lists by grouping the data based on member number and transaction date and store it in a data frame.

grouped_list_data = data.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).reset_index(name='Transaction Items')
print('Shape of the Data:', grouped_list_data.shape)
print("\nGrouped Data:")
#pd.set_option('display.max_colwidth', 2)
#print(grouped_data)
grouped_list_data.head()

Shape of the Data: (14963, 3)

Grouped Data:


Member_number        Date  \
0           1000  15-03-2015   
1           1000  24-06-2014   
2           1000  24-07-2015   
3           1000  25-11-2015   
4           1000  27-05-2015   

                                   Transaction Items  
0  [sausage, whole milk, semi-finished bread, yog...  
1                  [whole milk, pastry, salty snack]  
2                     [canned beer, misc. beverages]  
3                        [sausage, hygiene articles]  
4                         [soda, pickled vegetables]

### Task C: 7 points

Transform the data having unique product as column name and indication of the product in the each transaction as 0/1.

We are performing a transformation on the grouped transaction data to create a matrix-like representation where each row corresponds to a transaction, each column corresponds to a unique product, and the values indicate whether that product was present (1) or not (0) in each transaction.

In [5]:
# Transform the data to have unique products as column names and indications of the product in each transaction as 0 or 1.

encoded_data = grouped_list_data['Transaction Items'].apply(lambda x: pd.Series({item: 1 for item in x}))
encoded_data = encoded_data.fillna(0).astype(np.int8)
encoded_data['Member_number'] = grouped_list_data['Member_number']
encoded_data['Date'] = grouped_list_data['Date']
print("\nEncoded Data:")
encoded_data.head()


Encoded Data:


sausage  whole milk  semi-finished bread  yogurt  pastry  salty snack  \
0        1           1                    1       1       0            0   
1        0           1                    0       0       1            1   
2        0           0                    0       0       0            0   
3        1           0                    0       0       0            0   
4        0           0                    0       0       0            0   

   canned beer  misc. beverages  hygiene articles  soda  ...  decalcifier  \
0            0                0                 0     0  ...            0   
1            0                0                 0     0  ...            0   
2            1                1                 0     0  ...            0   
3            0                0                 1     0  ...            0   
4            0                0                 0     1  ...            0   

   kitchen utensil  organic products  frozen chicken  salad dressing  \
0                0                 0               0               0   
1                0                 0               0               0   
2                0                 0               0               0   
3                0                 0               0               0   
4                0                 0               0               0   

   specialty vegetables  toilet cleaner  rubbing alcohol  Member_number  \
0                     0               0                0           1000   
1                     0               0                0           1000   
2                     0               0                0           1000   
3                     0               0                0           1000   
4                     0               0                0           1000   

         Date  
0  15-03-2015  
1  24-06-2014  
2  24-07-2015  
3  25-11-2015  
4  27-05-2015  

[5 rows x 169 columns]

### Task D: 7 points

Apply apriori algorithm with 0.001 support

By applying the Apriori algorithm with a given support threshold, we are essentially identifying sets of items that frequently occur together in the transactions recorded in the encoded_data DataFrame.

In [6]:
# Apply the Apriori algorithm with a support threshold of 0.001.

support_threshold = 0.001
most_occurence_itemsets = apriori(encoded_data.drop(['Member_number', 'Date'], axis=1), min_support=support_threshold, use_colnames=True)
most_occurence_itemsets

/Users/vijithakotapati/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


support                                    itemsets
0    0.060349                                   (sausage)
1    0.157923                                (whole milk)
2    0.009490                       (semi-finished bread)
3    0.085879                                    (yogurt)
4    0.051728                                    (pastry)
..        ...                                         ...
745  0.001136      (yogurt, other vegetables, whole milk)
746  0.001002              (soda, rolls/buns, whole milk)
747  0.001136        (soda, other vegetables, whole milk)
748  0.001203  (other vegetables, rolls/buns, whole milk)
749  0.001136        (soda, rolls/buns, other vegetables)

[750 rows x 2 columns]

### Task E: 5 points

Print top 5 association rules for lift metric and 1 threshold in accordance with support value.

The code snippet below calculates association rules based on frequent itemsets, selects the top 5 rules with the highest lift values, and displays these rules along with their metrics.

In [7]:
# Print the top 5 association rules based on the lift metric and a threshold of 1, according to the support value.

ass_rules_df = association_rules(most_occurence_itemsets, metric="lift", min_threshold=1)
top_5_ass_rules = ass_rules_df.nlargest(5, 'lift')
print("\nTop 5 Association Rules based on Lift:")
top_5_ass_rules


Top 5 Association Rules based on Lift:


antecedents            consequents  antecedent support  \
211   (yogurt, whole milk)              (sausage)            0.011161   
214              (sausage)   (yogurt, whole milk)            0.060349   
212  (sausage, whole milk)               (yogurt)            0.008955   
213               (yogurt)  (sausage, whole milk)            0.085879   
166  (specialty chocolate)         (citrus fruit)            0.015973   

     consequent support   support  confidence      lift  leverage  conviction  \
211            0.060349  0.001470    0.131737  2.182917  0.000797    1.082219   
214            0.011161  0.001470    0.024363  2.182917  0.000797    1.013532   
212            0.085879  0.001470    0.164179  1.911760  0.000701    1.093681   
213            0.008955  0.001470    0.017121  1.911760  0.000701    1.008307   
166            0.053131  0.001403    0.087866  1.653762  0.000555    1.038081   

     zhangs_metric  
211       0.548014  
214       0.576701  
212       0.481231  
213       0.521727  
166       0.401735

### Task F: 3 points

Store the association rules from top to bottom with respect to lift parameter and store it in a variable.

The purpose of this code is to sort the association rules based on the 'lift' parameter, which is a measure of the strength of the association between the antecedent and consequent of a rule. By sorting the rules in descending order of lift, you can identify the most significant and potentially interesting rules that exhibit strong relationships between items. 

In [8]:
# Store the association rules from top to bottom, sorted by the lift parameter, and store them in a variable.

sorted_ass_rules = ass_rules_df.sort_values(by='lift', ascending=False)
sorted_ass_rules

antecedents              consequents  antecedent support  \
211     (yogurt, whole milk)                (sausage)            0.011161   
214                (sausage)     (yogurt, whole milk)            0.060349   
212    (sausage, whole milk)                 (yogurt)            0.008955   
213                 (yogurt)    (sausage, whole milk)            0.085879   
167           (citrus fruit)    (specialty chocolate)            0.053131   
..                       ...                      ...                 ...   
62                  (grapes)                   (soda)            0.014436   
119           (citrus fruit)                   (beef)            0.053131   
118                   (beef)           (citrus fruit)            0.033950   
106             (rolls/buns)  (fruit/vegetable juice)            0.110005   
107  (fruit/vegetable juice)             (rolls/buns)            0.034017   

     consequent support   support  confidence      lift      leverage  \
211            0.060349  0.001470    0.131737  2.182917  7.967480e-04   
214            0.011161  0.001470    0.024363  2.182917  7.967480e-04   
212            0.085879  0.001470    0.164179  1.911760  7.012151e-04   
213            0.008955  0.001470    0.017121  1.911760  7.012151e-04   
167            0.015973  0.001403    0.026415  1.653762  5.548137e-04   
..                  ...       ...         ...       ...           ...   
62             0.097106  0.001403    0.097222  1.001195  1.674919e-06   
119            0.033950  0.001804    0.033962  1.000349  6.297697e-07   
118            0.053131  0.001804    0.053150  1.000349  6.297697e-07   
106            0.034017  0.003743    0.034022  1.000136  5.091755e-07   
107            0.110005  0.003743    0.110020  1.000136  5.091755e-07   

     conviction  zhangs_metric  
211    1.082219       0.548014  
214    1.013532       0.576701  
212    1.093681       0.481231  
213    1.008307       0.521727  
167    1.010726       0.417500  
..          ...            ...  
62     1.000129       0.001211  
119    1.000012       0.000369  
118    1.000020       0.000361  
106    1.000005       0.000153  
107    1.000017       0.000141  

[240 rows x 10 columns]

### Task G: 10 points

Understand this,
- antecedent support: If X is called antecendent, 'antecedent support' computes the proportion of transactions that contain the antecedent X.
- consequent support: If Y is called consequent, 'consequent support' computes the proportion of transactions that contain the antecedent Y.
- support: 'support' computes the proportion of transactions that contain the antecedent X and Y.
- confidence: Probability of buying Y when X is bought.
- lift: Represents how many times the probability of getting Y increases when X is received.

Define a function which takes the (sorted values from the association rules,a product name) and returns top recommendation list for that product name.

Tip: the recommendated item are Consequents by matching the product antecedents as product to search for.

In [9]:
# Define a function that takes the sorted association rules and a product name as inputs.
# The function should return the top recommendation list for that product name.

def get_top_recs(sorted_values, product_name):
    top_recs= sorted_values[sorted_values['antecedents'].apply(lambda x: product_name in x)]
    return top_recs

This function is used to retrieve the top recommendation list for a specific product based on the sorted association rules. It filters the association rules to find those where the given product name appears in the antecedents and returns these filtered rules as the top recommendations.

### Task H: 5 points

Print top 5 reccomendated items for product 'sausage'.

This code finds and display the top 5 recommended items for the product 'sausage' based on sorted association rules, assuming the get_top_recs() function is properly defined and returns the desired recommendations.

In [10]:
# Print the top 5 recommended items for the product 'sausage'.

item_name = 'sausage'
top_recs_for_sausage = get_top_recs(sorted_ass_rules, item_name)
print("\nTop 5 Recommended Items for 'sausage':")
top_recs_for_sausage.head()


Top 5 Recommended Items for 'sausage':


antecedents           consequents  antecedent support  \
214              (sausage)  (yogurt, whole milk)            0.060349   
212  (sausage, whole milk)              (yogurt)            0.008955   
210      (yogurt, sausage)          (whole milk)            0.005748   
25               (sausage)           (beverages)            0.060349   
219              (sausage)    (soda, whole milk)            0.060349   

     consequent support   support  confidence      lift  leverage  conviction  \
214            0.011161  0.001470    0.024363  2.182917  0.000797    1.013532   
212            0.085879  0.001470    0.164179  1.911760  0.000701    1.093681   
210            0.157923  0.001470    0.255814  1.619866  0.000563    1.131541   
25             0.016574  0.001537    0.025471  1.536764  0.000537    1.009129   
219            0.011629  0.001069    0.017719  1.523708  0.000368    1.006200   

     zhangs_metric  
214       0.576701  
212       0.481231  
210       0.384877  
25        0.371714  
219       0.365781

### Task I: 5 points

Print top 5 reccomendated items for product 'sausage' with 0.005 support.

The code calculates and displays the top 5 recommended items to purchase alongside the product 'sausage' based on association rules and a minimum support threshold of 0.005, which determines the frequency of occurrence of itemsets in the transactions. The recommendations are based on the lift metric, which indicates the strength of the association between items.

In [11]:
# Print the top 5 recommended items for the product 'sausage' with a support threshold of 0.005.

most_occurence_itemsets_sausage = apriori(encoded_data.drop(['Member_number', 'Date'], axis=1), min_support=0.005, use_colnames=True)
ass_rules_df_sausage = association_rules(most_occurence_itemsets_sausage, metric="lift", min_threshold=1)
sorted_ass_rules_sausage = ass_rules_df_sausage.sort_values(by='lift', ascending=False)
top_recs_sausage_new_threshold = get_top_recs(sorted_ass_rules_sausage, item_name)
print("\nTop 5 Recommended Items for 'sausage' with Support Threshold 0.005:")
top_recs_sausage_new_threshold.head()


Top 5 Recommended Items for 'sausage' with Support Threshold 0.005:


/Users/vijithakotapati/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


antecedents consequents  antecedent support  consequent support   support  \
1   (sausage)    (yogurt)            0.060349            0.085879  0.005748   
2   (sausage)      (soda)            0.060349            0.097106  0.005948   

   confidence      lift  leverage  conviction  zhangs_metric  
1    0.095238  1.108986  0.000565    1.010345       0.104587  
2    0.098560  1.014975  0.000088    1.001613       0.015702

### Task J: 10 points

Perform task D through I except task G using FP-Growth algorithm.

In [12]:
#TASK D: Apply apriori algorithm with 0.001 support

from mlxtend.frequent_patterns import fpgrowth
import pandas as pd
import numpy as np

support_threshold = 0.001

#Group the data and create a list of transactions
grouped_list_data = data.groupby(['Member_number', 'Date'])['itemDescription'].apply(list).reset_index(name='Transaction Items')

# Convert the list of transactions to a DataFrame with binary encoding
all_items_binary = set(item for sublist in grouped_list_data['Transaction Items'] for item in sublist)

def encode_items(items):
    itemset = set(items)
    return pd.Series({item: 1 if item in itemset else 0 for item in all_items_binary})

encoded_data = grouped_list_data['Transaction Items'].apply(encode_items)

#Get the frequent itemsets using FP-Growth algorithm
frequent_itemsets = fpgrowth(encoded_data, min_support=support_threshold, use_colnames=True)
frequent_itemsets

#References : https://stackoverflow.com , 

/Users/vijithakotapati/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


support                         itemsets
0    0.157923                     (whole milk)
1    0.085879                         (yogurt)
2    0.060349                        (sausage)
3    0.009490            (semi-finished bread)
4    0.051728                         (pastry)
..        ...                              ...
745  0.001403            (chewing gum, yogurt)
746  0.001069  (chewing gum, other vegetables)
747  0.001002              (chewing gum, soda)
748  0.001069              (pasta, whole milk)
749  0.001002  (seasonal products, rolls/buns)

[750 rows x 2 columns]

In [13]:
#TASK E : Print top 5 association rules for lift metric and 1 threshold in accordance with support value.

from mlxtend.frequent_patterns import fpgrowth, association_rules
association_rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

#Filter and print the top 5 rules based on lift
top_5_rules = association_rules_df.sort_values(by='lift', ascending=False).head(5)
top_5_rules

# References : https://www.tutorialspoint.com/python/index.html

antecedents            consequents  antecedent support  \
11   (yogurt, whole milk)              (sausage)            0.011161   
14              (sausage)   (yogurt, whole milk)            0.060349   
12  (sausage, whole milk)               (yogurt)            0.008955   
13               (yogurt)  (sausage, whole milk)            0.085879   
94  (specialty chocolate)         (citrus fruit)            0.015973   

    consequent support   support  confidence      lift  leverage  conviction  \
11            0.060349  0.001470    0.131737  2.182917  0.000797    1.082219   
14            0.011161  0.001470    0.024363  2.182917  0.000797    1.013532   
12            0.085879  0.001470    0.164179  1.911760  0.000701    1.093681   
13            0.008955  0.001470    0.017121  1.911760  0.000701    1.008307   
94            0.053131  0.001403    0.087866  1.653762  0.000555    1.038081   

    zhangs_metric  
11       0.548014  
14       0.576701  
12       0.481231  
13       0.521727  
94       0.401735

In [14]:
#TASK F : Store the association rules from top to bottom with respect to lift parameter and store it in a variable.

sorted_rules = association_rules_df.sort_values(by='lift', ascending=False)

# Store the sorted rules in a variable
association_rules_top_to_bottom = sorted_rules.copy()

# Print the top 5 rules (optional)
top_5_rules = sorted_rules.head(5)
top_5_rules

antecedents            consequents  antecedent support  \
11   (yogurt, whole milk)              (sausage)            0.011161   
14              (sausage)   (yogurt, whole milk)            0.060349   
12  (sausage, whole milk)               (yogurt)            0.008955   
13               (yogurt)  (sausage, whole milk)            0.085879   
94  (specialty chocolate)         (citrus fruit)            0.015973   

    consequent support   support  confidence      lift  leverage  conviction  \
11            0.060349  0.001470    0.131737  2.182917  0.000797    1.082219   
14            0.011161  0.001470    0.024363  2.182917  0.000797    1.013532   
12            0.085879  0.001470    0.164179  1.911760  0.000701    1.093681   
13            0.008955  0.001470    0.017121  1.911760  0.000701    1.008307   
94            0.053131  0.001403    0.087866  1.653762  0.000555    1.038081   

    zhangs_metric  
11       0.548014  
14       0.576701  
12       0.481231  
13       0.521727  
94       0.401735

In [15]:
#TASK H : Print top 5 reccomendated items for product 'sausage'.

recommended_rows = sorted_rules[sorted_rules['antecedents'].apply(lambda x: 'sausage' in x)].head(5)
recommended_rows

antecedents           consequents  antecedent support  \
14               (sausage)  (yogurt, whole milk)            0.060349   
12   (sausage, whole milk)              (yogurt)            0.008955   
10       (yogurt, sausage)          (whole milk)            0.005748   
211              (sausage)           (beverages)            0.060349   
26               (sausage)    (soda, whole milk)            0.060349   

     consequent support   support  confidence      lift  leverage  conviction  \
14             0.011161  0.001470    0.024363  2.182917  0.000797    1.013532   
12             0.085879  0.001470    0.164179  1.911760  0.000701    1.093681   
10             0.157923  0.001470    0.255814  1.619866  0.000563    1.131541   
211            0.016574  0.001537    0.025471  1.536764  0.000537    1.009129   
26             0.011629  0.001069    0.017719  1.523708  0.000368    1.006200   

     zhangs_metric  
14        0.576701  
12        0.481231  
10        0.384877  
211       0.371714  
26        0.365781

In [16]:
#TASK I : Print top 5 reccomendated items for product 'sausage' with 0.005 support.

support_threshold = 0.005

# Get the frequent itemsets using FP-Growth algorithm with the adjusted support threshold
frequent_itemsets = fpgrowth(encoded_data, min_support=support_threshold, use_colnames=True)

# Generate association rules
association_rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Sort the rules based on lift in descending order
sorted_rules = association_rules_df.sort_values(by='lift', ascending=False)

# Filter the rules for 'sausage' in the antecedents and get the top 5 recommended rows
recommended_rows = sorted_rules[sorted_rules['antecedents'].apply(lambda x: 'sausage' in x)].head(5)

recommended_rows

#References : https://www.geeksforgeeks.org/python-programming-language/ , https://www.tutorialspoint.com/python/index.html

/Users/vijithakotapati/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


antecedents consequents  antecedent support  consequent support   support  \
1   (sausage)    (yogurt)            0.060349            0.085879  0.005748   
3   (sausage)      (soda)            0.060349            0.097106  0.005948   

   confidence      lift  leverage  conviction  zhangs_metric  
1    0.095238  1.108986  0.000565    1.010345       0.104587  
3    0.098560  1.014975  0.000088    1.001613       0.015702

### Task K: 8 points

Print Top 3 recommendations for below products using Apriori with 0.0015 support
- citrus fruit
- whole milk
- soda

In [17]:
# Print the top 3 recommendations for the following products using the Apriori algorithm with a support threshold of 0.0015:
# 1. citrus fruit
# 2. whole milk
# 3. soda

items_recommended = ['citrus fruit', 'whole milk', 'soda']
top_recs_products = []

for item in items_recommended:
    top_recs_products.append(get_top_recs(sorted_ass_rules, item).head(3))

print("\nTop 3 Recommendations for 'citrus fruit', 'whole milk', and 'soda':")


Top 3 Recommendations for 'citrus fruit', 'whole milk', and 'soda':


In [18]:
print("\nProduct:", items_recommended[0])
top_recs_products[0]


Product: citrus fruit


antecedents            consequents  antecedent support  \
167  (citrus fruit)  (specialty chocolate)            0.053131   
205  (citrus fruit)                (candy)            0.053131   
206  (citrus fruit)              (napkins)            0.053131   

     consequent support   support  confidence      lift  leverage  conviction  \
167            0.015973  0.001403    0.026415  1.653762  0.000555    1.010726   
205            0.014369  0.001002    0.018868  1.313120  0.000239    1.004586   
206            0.022121  0.001403    0.026415  1.194106  0.000228    1.004410   

     zhangs_metric  
167       0.417500  
205       0.251835  
206       0.171675

In [19]:
print("\nProduct:", items_recommended[1])
top_recs_products[1]


Product: whole milk


antecedents        consequents  antecedent support  \
211   (yogurt, whole milk)          (sausage)            0.011161   
212  (sausage, whole milk)           (yogurt)            0.008955   
215           (whole milk)  (yogurt, sausage)            0.157923   

     consequent support  support  confidence      lift  leverage  conviction  \
211            0.060349  0.00147    0.131737  2.182917  0.000797    1.082219   
212            0.085879  0.00147    0.164179  1.911760  0.000701    1.093681   
215            0.005748  0.00147    0.009310  1.619866  0.000563    1.003596   

     zhangs_metric  
211       0.548014  
212       0.481231  
215       0.454430

In [20]:
print("\nProduct:", items_recommended[2])
top_recs_products[2]


Product: soda


antecedents            consequents  antecedent support  \
218  (soda, whole milk)              (sausage)            0.011629   
64               (soda)                  (oil)            0.097106   
220              (soda)  (sausage, whole milk)            0.097106   

     consequent support   support  confidence      lift  leverage  conviction  \
218            0.060349  0.001069    0.091954  1.523708  0.000368    1.034806   
64             0.014903  0.001804    0.018582  1.246844  0.000357    1.003748   
220            0.008955  0.001069    0.011012  1.229612  0.000200    1.002079   

     zhangs_metric  
218       0.347750  
64        0.219267  
220       0.206819

### Task L: 8 points

Print Top 3 recommendations for below products using FP-Growth algorithm with 0.0015 support
- canned beer
- yogurt
- pastry

In [21]:
support_threshold = 0.0015 
frequent_itemsets = fpgrowth(encoded_data, min_support=support_threshold, use_colnames=True)

# Generate association rules
association_rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Filter the rules for each product and get the top 3 recommended rows
top_3_recommended_canned_beer = association_rules_df[association_rules_df['antecedents'].apply(lambda x: 'canned beer' in x)].nlargest(3, 'lift')
top_3_recommended_yogurt = association_rules_df[association_rules_df['antecedents'].apply(lambda x: 'yogurt' in x)].nlargest(3, 'lift')
top_3_recommended_pastry = association_rules_df[association_rules_df['antecedents'].apply(lambda x: 'pastry' in x)].nlargest(3, 'lift')

print("Top 3 recommendations for 'canned beer':")
top_3_recommended_canned_beer

Top 3 recommendations for 'canned beer':


/Users/vijithakotapati/opt/anaconda3/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


antecedents    consequents  antecedent support  consequent support  \
20  (canned beer)  (white bread)            0.046916            0.023993   
52  (canned beer)  (brown bread)            0.046916            0.037626   
45  (canned beer)   (newspapers)            0.046916            0.038896   

     support  confidence      lift  leverage  conviction  zhangs_metric  
20  0.001537    0.032764  1.365573  0.000411    1.009068       0.280885  
52  0.002406    0.051282  1.362937  0.000641    1.014394       0.279398  
45  0.001871    0.039886  1.025455  0.000046    1.001031       0.026045

In [22]:
print("\nTop 3 recommendations for 'yogurt':")
top_3_recommended_yogurt


Top 3 recommendations for 'yogurt':


antecedents     consequents  antecedent support  consequent support  \
0     (yogurt)       (sausage)            0.085879            0.060349   
48    (yogurt)  (citrus fruit)            0.085879            0.053131   

     support  confidence      lift  leverage  conviction  zhangs_metric  
0   0.005748    0.066926  1.108986  0.000565    1.007049       0.107508  
48  0.004611    0.053696  1.010642  0.000049    1.000598       0.011520

In [23]:
print("\nTop 3 recommendations for 'pastry':")
top_3_recommended_pastry

#Reference: https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/


Top 3 recommendations for 'pastry':


antecedents    consequents  antecedent support  consequent support  \
58    (pastry)      (napkins)            0.051728            0.022121   
50    (pastry)  (brown bread)            0.051728            0.037626   
6     (pastry)      (sausage)            0.051728            0.060349   

     support  confidence      lift  leverage  conviction  zhangs_metric  
58  0.001738    0.033592  1.518529  0.000593    1.011869       0.360095  
50  0.002005    0.038760  1.030127  0.000059    1.001179       0.030841  
6   0.003208    0.062016  1.027617  0.000086    1.001777       0.028341

### Task M: 20 points

Submit a document showing your solution for below problem.

For the data on apriori image in the folder, apply apriori algorithm by

- Computing the support for each individual item
- Use 7 as the minimum support.
- Generate Association Rules and compute confidence 
- use min confidence as 85%.

#### Please refer the the document "Khatri_9464_Kanneti_2500_Kotapati_0730_Task_M "

### Programming Assignment Details

1. If using any resource (books, internet), please make sure that you cite it within that cell.
2. Do not rename the dataset_files.

###  Submission details
Fill your name and ID in the jupyter notebook for each group member in the following format:

1. First Student Name and ID: ABC 1001XXXXXX
2. Second Student Name and ID: DEF 1002XXXXXX
3. Third Student Name and ID: GHI 1003XXXXXX

Name your submission files:

yourLastName_Last4digitsofyourID.ipynb

EG: abc_1234_def_5678_xyz3819_python.ipynb

### NOTE: Only one team member will submit the file.